# Google OR-Tools 测试

## 测试说明

### 官方页面

https://developers.google.com/optimization/routing/cvrp

### 算法的输入
    - 两个地点之间的交通时间
    - 每个门店的需求量
    - 每辆货车的最大承载能力
### 限制条件
    - 所有门店的需求量的总和不能大于所有货车的最大承载能力总和
    - 单家门店的需求量不能大于最大单辆货车承载能力
    - 时间、需求量、承载能力均为Integer

## 运行过程

### 安装依赖包

见`requirements.txt`

### 运行程序
    - 数据存储在`./data`文件夹下
    - 时间矩阵(`distance_matrix`)使用`./gaode/distance_matrix_20221227_1657.csv`
    - 数据中假设存在1个仓库与111个门店
    - 门店需求和货车数据为随机生成, 生成代码见`./utils/data_generator.py`
    - 假设门店需求范围为 [1, 30]
    - 一共有22辆货车, 每辆车运载能力为 [25, 150]

In [58]:
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
import pandas as pd
from IPython.display import display

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.genfromtxt('./data/distance_matrix.csv', delimiter=',', dtype=int)
    data['demands'] = np.genfromtxt('./data/demands.csv', delimiter=',', dtype=int)
    data['vehicle_capacities'] = np.genfromtxt('./data/vehicle_capacities.csv', delimiter=',', dtype=int)
    data['num_vehicles'] = len(data['vehicle_capacities'])
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution and plot."""
    locations = pd.read_csv("./gaode/locations.csv")
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    trip_df = pd.DataFrame()
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(locations['name'][node_index], route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            trip_df = pd.concat([trip_df, pd.DataFrame({'vehicle_id': [vehicle_id], 'origin': [node_index], 'destination': [manager.IndexToNode(index)]})], ignore_index=True)
        plan_output += ' {0} Load({1})\n'.format(locations['name'][manager.IndexToNode(index)],
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    
    # make folium plot
    category_20 = ('#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', 
                   '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5')
    trip_df = trip_df.query('origin!=0 | destination!=0 ')
    merged_trip_locations = pd.merge(trip_df, locations[['location', 'name']], left_on='origin', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "origin_location", 'name': 'origin_name'}, inplace=True)
    merged_trip_locations = pd.merge(merged_trip_locations, locations[['location', 'name']], left_on='destination', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "destination_location", 'name': 'destination_name'}, inplace=True)
    merged_trip_locations['origin_location'] = merged_trip_locations['origin_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['destination_location'] = merged_trip_locations['destination_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['color'] = merged_trip_locations['vehicle_id'].map(dict(zip(merged_trip_locations['vehicle_id'].unique(), category_20)))

    trip_map = folium.Map([24.48, 118.08], 
                          tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7',
                          attr='高德-常规图',
                          zoom_start=13,)

    for _, row in merged_trip_locations.iterrows():
        folium.CircleMarker(row['origin_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['origin_name'], 
                           ).add_to(trip_map)
        
        folium.CircleMarker(row['destination_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['destination_name'], 
                           ).add_to(trip_map)
    
        folium.PolyLine([row['origin_location'], row['destination_location']],
                        color=row['color'], 
                       ).add_to(trip_map)
    display(trip_map)

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 1736
Route for vehicle 0:
 厦门鹭燕制药有限公司 Load(0) ->  厦门鹭燕制药有限公司 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 厦门鹭燕制药有限公司 Load(0) ->  厦门鹭燕制药有限公司 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 厦门鹭燕制药有限公司 Load(0) ->  厦门鹭燕制药有限公司 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 3:
 厦门鹭燕制药有限公司 Load(0) ->  燕鹭大药房(巷南路店) Load(18) ->  鹭燕大药房(马巷五星分店) Load(44) ->  鹭燕大药房(巷南分店) Load(59) ->  鹭燕大药房(马巷分店) Load(79) ->  鹭燕大药房(鼓锣东六里店) Load(92) ->  鹭燕大药房(新兴分店) Load(111) ->  鹭燕大药房(新店分店) Load(113) ->  鹭燕大药房(翔安店) Load(123) ->  鹭燕大药房(城场分店) Load(141) ->  厦门鹭燕制药有限公司 Load(141)
Distance of the route: 142m
Load of the route: 141

Route for vehicle 4:
 厦门鹭燕制药有限公司 Load(0) ->  鹭燕大药房(屿后分店) Load(6) ->  鹭燕大药房(东坪山路分店) Load(17) ->  鹭燕大药房(文园路分店) Load(27) ->  鹭燕大药房(镇海分店) Load(43) ->  鹭燕大药房(镇海路分店) Load(63) ->  鹭燕大药房(霞溪路店) Load(71) ->  鹭燕大药房(镇海路新特药店) Load(74) ->  鹭燕大药房(镇海路三分店) Load(100) ->  厦门鹭燕制药有限公司 Load(100)
Distance of the route: 

### 结果评估

    - 与现有运输方案比较
        - 总运输时间
        - 运载率: 派出车辆的总运输量/总可承载量
    - 应用细节问题
        - 在门店的装卸货时间是否应该加入运输时间(按装载量估算, 或者如果门店处有记载也可以按门店计算时间与需求关系)
        - 门店需求对应的装箱体积是否可以估算
            - 虽然仓储分装好后有一个确定的数据,但是从流程优化的角度应该是分拣装载好一辆货车的货物后再进行下一辆车的装载, 因此不应该等到所有门店货物全部装箱完毕后再分配载货货车
        - 现有物流方案是固定运输线路, 每天满足部分门店需求, 是否可以比较算法选出的运输线路与现有固定线路, 并与熟悉路况的司机探讨, 找出优化线路
        